In [2]:
import pandas as pd

dataframe = pd.read_csv(
    "https://docs.dagster.io/assets/iris.csv",
    names=[
        "sepal_length_cm",
        "sepal_width_cm",
        "petal_length_cm",
        "petal_width_cm",
        "species",
    ],
)

dataframe.values[0:10]

array([[5.1, 3.5, 1.4, 0.2, 'Iris-setosa'],
       [4.9, 3.0, 1.4, 0.2, 'Iris-setosa'],
       [4.7, 3.2, 1.3, 0.2, 'Iris-setosa'],
       [4.6, 3.1, 1.5, 0.2, 'Iris-setosa'],
       [5.0, 3.6, 1.4, 0.2, 'Iris-setosa'],
       [5.4, 3.9, 1.7, 0.4, 'Iris-setosa'],
       [4.6, 3.4, 1.4, 0.3, 'Iris-setosa'],
       [5.0, 3.4, 1.5, 0.2, 'Iris-setosa'],
       [4.4, 2.9, 1.4, 0.2, 'Iris-setosa'],
       [4.9, 3.1, 1.5, 0.1, 'Iris-setosa']], dtype=object)

In [20]:
from sqlalchemy import create_engine
from sqlalchemy import text

import fsspec
import os
from deltalake import write_deltalake

engine = create_engine(
    "trino://trino@localhost:8080/delta", connect_args={"user": "trino"}
)

conn = engine.connect()
cur = conn.execute(text("CREATE SCHEMA IF NOT EXISTS notebook WITH (LOCATION='s3a://warehouse/delta-notebook/')"))

# https://s3fs.readthedocs.io/en/latest/api.html#s3fs.core.S3FileSystem
fs = fsspec.filesystem(protocol="s3a", key="minio", secret="minio123", endpoint_url="http://localhost:4566")

file_path = os.path.join("delta-notebook", "iris")
fs.makedirs(os.path.dirname(file_path), exist_ok=True)

write_deltalake(file_path, dataframe, mode='overwrite')
fs.put(file_path, "s3a://warehouse/delta-notebook/iris", recursive=True)

[None, None]

In [24]:
from sqlalchemy import text


with engine.connect() as conn:
    conn.execute(text(f"""
        CALL delta.system.register_table(schema_name => 'notebook', table_name => 'iris', table_location => 's3a://warehouse/delta-notebook/iris')
    """))
    conn.commit()
